#  🍔 요기요 데이터 수집

## 사전 자체 데이터를 수집하기 위해 요기요 웹페이지 대상으로 크롤링을 수행한다. 
### 수집하는 과정은 다음과 같다.
- 리뷰 데이터 수집을 위한 사전단계로서 가게별 URL 수집
- 가게별 URL 기반 리뷰 데이터 수집
- 추가 분석을 위한 가게 평점, 메뉴 등 전반적인 가게 정보 수집


# 1. 요기요 가게별 URL 수집

## 개요
요기요 홈페이지는 대부분의 요소가 동적 페이지로 구성되어 있다. 이에 따라 발생하는 이슈를 해결하며, 데이터 수집의 효율성을 위해 가게별 접근이 직접적으로 가능하도록 리뷰 데이터에 수집에 앞서 가게별로 URL을 수집한다. 


- 먼저 URL을 수집한 첫번째 이유는 하나의 가게 리뷰를 수집한 후 기존의 가게 리스트로 돌아가게 될 경우 동적으로 요소들이 로딩되어 기존의 가게 리스트 순서가 보장되지 않는 문제를 해결하기 위함이다. 가게 리스트의 순서가 보장되지 않는 이유는 시간대에 따라 현재 오픈 중인 가게가 동적으로 리스트 상위에 위치하게 되기 때문이며, 수집하는 양과 소요되는 시간을 고려하였을 때 중복 또는 누락되는 가게가 생길 가능성이 크다. 
 
 

- 두번째 이유는 요기요의 가게 리스트가 페이지 마지막 스크롤마다 60개씩 새로 생성됨에 따라 발생하는 비효율성을 해결하기 위함이다. 요기요 페이지는 하나의 카테고리당 많게는 수백, 수천개의 가게가 존재한다. 이에 하단에 위치하는 가게에 접근하기 위해서는 상당히 많은 횟수의 스크롤이 필요하며, 이 과정에서 네트워크 지연, 인터넷 사용기록의 과다한 적재 등의 문제가 발생하였을 때는 예외처리로도 복구하기 어려운 작업 손실과 시간 소요가 발생한다. 특히 첫번째 이유에서 언급했듯 가게 리스트가 동적으로 요소들이 재로딩됨에 따라 하나의 가게 수집 후 다시 가게 리스트로 돌아오는 경우 다시 첫번째 가게부터 다음 수집 대상 가게까지 스크롤을 해야하는 상황이 벌어진다. 이는 후반부의 가게로 갈수록 시간이 급격하게 늘어나 큰 비효율을 야기하게 된다.
 
 
- 이에 초기에 URL을 미리 수집해둔 후 가게별로 접근할 때 사용함으로써 큰 시간 절감 효과를 얻을 수 있다.





### 라이브러리 import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from tqdm import tqdm
import pandas as pd
import time
import datetime

### 카테고리별 가게 개수 수집

수집을 원하는 지역과 카테고리를 입력 받아 해당 지역으로 지역설정을 변경한다. 이후 해당 카테고리에 접속하여 리뷰 많은 순으로 정렬 후 스크롤을 끝까지 내려 해당 카테고리 내에 가게가 몇개 존재하는지 반환한다.

In [2]:
def num_of_reviews(location, food_category):
    location
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    urls = []
    df = pd.DataFrame(columns=['url', '개수충족'])

    #### 주소 기준으로 초기화
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(8)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(location)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(2)
    
    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(3)
    
    # 카테고리 접속 및 가게 리스트 
    driver.get(list_url + food_category)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)

    while True:

        scroll_down = 0
        while scroll_down < 10:
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            scroll_down += 1

        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(1)
        if new_height == last_height:
            print("끝")
            break

        last_height = new_height
    
    store_list = driver.find_elements_by_xpath("//div[@class='item clearfix']")
    
    return len(store_list)

### 가게별 URL 수집

수집을 원하는 지역, 카테고리, 시작번호, 끝번호를 인자로 받아 URL 정보를 수집한다. 이 때 href 태그를 통한 URL 수집이 불가능하므로 하나의 가게에 직접 접속 후 다시 되돌아가는 작업이 필요하게 되고, 앞서 개요에서 언급한 두번째 단점을 수반하는 것이 불가피하다. 이로 인해 발생하는 수많은 스크롤 조작에 의한 부하를 방지하기 위하여 100개씩 URL을 수집하며, 이를 위하여 앞서 수집한 카테고리별 가게 개수 데이터를 활용한다. 

이 때 수집되는 데이터는 
- 가게별 URL
- 가게명
- 리뷰개수 충족 여부이다. 

In [3]:
def get_url3(location, food_category, start, end):
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    urls = []
    parsed_data = pd.read_csv("./강남구url/강남구_"+food_category+"_url.csv")
    stores = parsed_data['가게명'].tolist()
    df = pd.DataFrame(columns=['url', '개수충족','가게명'])
    
    #### 주소 기준으로 초기화
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(8)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(location)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(2)
    
    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(3)
    
    # 카테고리 접속 및 가게 리스트 
    driver.get(list_url + food_category) 
    
    last_height = driver.execute_script("return document.body.scrollHeight")


    try:
        for i in tqdm(range(start, end)):
            time.sleep(0.5)
            is_valid = 1
            path = '//*[@id="content"]/div/div[5]/div/div/div[' + str(i) + ']/div'
            
            # 가게명 요소 찾기(스크롤이 필요 없는 초기 60개 리스트)
            try:
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
            
            # 지정 횟수만큼 스크롤    
            except:
                for j in range(int(i // 60)):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    if i<480:
                        time.sleep(i//60)
                    else:
                        time.sleep(8)
            
            # 가게명과 리뷰 개수 찾기           
            try:
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
            
            # 찾지 못한다면 네트워크 부하로 아직 정보가 나타나지 않았으므로 대기 및 재스크롤 후 요소 찾음 
            except:
                time.sleep(6)
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
            
            # 이미 중복된 가게라면 넘어가기
            print(store_name)
            if store_name in stores:
                print("skip")
                continue
             
            # 리뷰 개수가 10개 이상인지 확인
            some_tag = driver.find_element_by_xpath(path)
            if n == "" or int(n.split(" ")[1]) < 10:
                is_valid = 0

            # somthing element 까지 스크롤
            # 해당 페이지 접속
            action = ActionChains(driver)
            action.move_to_element(some_tag).perform()
            driver.find_element_by_xpath(path).click()
            time.sleep(2)

            row = pd.DataFrame([(driver.current_url, is_valid, store_name)],columns=['url', '개수충족','가게명'])
            print(row)
            df = df.append(row)

            driver.back()
            time.sleep(3)
    
    # 예외처리로 해결할 수 없는 문제 발생 시 현재까지의 정보 저장 
    except:
        df.to_csv("./temp/"+location.split(" ")[1]+"_"+food_category+"_url_list2_비정상종료-"+str(start)+".csv",
              encoding='utf-8-sig')
        
    # 결과 저장    
    df.to_csv("./temp/"+location.split(" ")[1]+"_"+food_category+"_url_list2-"+str(start)+".csv",
              encoding='utf-8-sig')
    driver.close()

### 가게 이름 수집

초기 URL 수집 메소드는 이름을 저장하지 않아 수집된 URL에 접속해 가게명을 추출하는 과정을 거쳤다. 위의 get_url3로 변경된 후 사용하지 않는다.

In [5]:
def get_name(category):
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')
    df = pd.read_csv("./강남구_"+category+"_url_list.csv")
    urls = df['url'].tolist()
    store_name = []
    for url in urls:
        driver.get(url)
        time.sleep(1.5)
        try:
            store_name.append(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text)
        except:
            print(url)
    df['가게명']=store_name
    df.to_csv("./강남구_"+category+"_url_list.csv", encoding='utf-8-sig')

### URL 데이터 수집

In [ ]:
# 카테고리별 가게 개수 수집 
dic = {}
categories = ["프랜차이즈", "한식", "치킨", "피자양식", "중식", "일식돈까스", "족발보쌈", "야식", "분식", "카페디저트", "편의점"]

for cat in categories:
    dic[cat] = num_of_reviews("서울특별시 강남구 삼성동 16-1 강남구청", cat)
    
'''
결과 

dic = {'1인분주문': 530,
 '프랜차이즈': 818,
 '치킨': 331,
 '피자양식': 571,
 '중식': 210,
 '한식': 1614,
 '일식돈까스': 607,
 '족발보쌈': 126,
 '야식': 677,
 '분식': 666,
 '카페디저트': 727,
 '편의점': 80}

'''

# 카테고리별로 url 수집
# 맨 아래까지 스크롤 시 60개씩 동적으로 가게 추가 표시되어 컴퓨터에 상당한 무리가 가며, 해당 가게에 접속하여 url 수집 후 되돌아오면 세션이 초기화 됨
# 에러 핸들링을 위하여 100개씩 나누어서 수집
for cat in categories:
    review_num = dic[cat]
    temp = review_num
    n = 1
    for i in range(review_num//100+1):
        print(n, temp)
        if temp < 100:
            get_url3("서울특별시 강남구 삼성동 16-1 강남구청", cat, n, review_num+1)
            break
        get_url3("서울특별시 강남구 삼성동 16-1 강남구청", cat, n, n+100)
        n+=100
        temp = review_num - n
        

# 수집된 url 데이터셋에 가게이름 추가
for category in categories:
    try:
        get_name(category)
    except:
        print(category)
        continue

# 2. 요기요 리뷰 크롤링
## 개요

앞서 수집한 URL 데이터를 기반으로 가게별 리뷰 데이터를 수집한다. 11개의 업종 카테고리에 대해서 기준에 충족하는 가게 페이지로 접속 후, 한 건의 리뷰에 대하여 다음과 같은 데이터를 수집한다.
- 시 
- 구
- 업종명
- 가게명
- 년
- 월
- 전체 평점
- 맛 평점
- 양 평점
- 배달 평점
- 리뷰 내역
- 주문 내역

### 리뷰 데이터 크롤링
지역과 카테고리를 인자로 받아 해당 지역/업종에 대해 URL 기반 리뷰를 수집한다.

In [6]:
def yogiyo_crawling(location, food_category):
    
    # url 가져오기
    urls = pd.read_csv("./"+location.split(" ")[1]+"_"+food_category+"_url_list.csv")
    
    # 유효한 가게(리뷰개수 10개 이상)만 가져오기 
    is_valid = urls['개수충족'] == 1
    urls = urls[is_valid]
    urls = urls['url'].values.tolist()
    
    address = location
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    
    # 결과 데이터셋 생성
    df = pd.DataFrame(columns=['시', '구', '업종명', '가게명', '년', '월', '전체평점', '맛 평점', '양 평점', '배달 평점', '리뷰 내용', '주문 내역'])

    try:
        # 가게별로 접속하여 리뷰 수집
        for url in tqdm(urls):
            # 해당 가게 url로 접속
            driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')
            driver.get(url)
            time.sleep(3)
            
            # 리뷰 개수
            num_of_review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]')
            
            # 가게명
            store_name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text
            
            # 리뷰를 볼 수 있는 버튼 찾고 클릭  
            try:
                review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            except:
                time.sleep(3)
                review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')

            driver.execute_script("arguments[0].click();", review)
            time.sleep(1)

            # 전체 리뷰 개수 확인    
            try:
                num_of_review = driver.find_element_by_xpath(
                    '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]')
                if num_of_review == '':
                    num_of_review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span')
            except:
                time.sleep(3)
                num_of_review = driver.find_element_by_xpath(
                    '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]')
                if num_of_review == '':
                    num_of_review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span')

                    
            # 동적으로 변하는 더보기 버튼의 인덱스 계산(더보기 클릭 시 10개씩 증가) 및 전체 리뷰 확인 가능할 때 까지 클릭      
            j = 0
            idx = 11
            max_idx = int(num_of_review.text) // 10

            while True:
                if j == max_idx:
                    html = driver.page_source
                    break
                try:
                    driver.find_element_by_css_selector('#review > li.list-group-item.btn-more > a').click()
                    time.sleep(1.5)

                    dates = driver.find_element_by_xpath(
                        "// *[ @ id = 'review'] / li[" + str(idx) + "] / div[1] / span[2]").text
                    if "전" in dates or "어제" in dates or int(dates.split("년")[0]) > 2019:
                        pass
                    elif int(dates.split("년")[0]) == 2019 and int(dates.split(" ")[1][:-1]) >= 3:
                        pass
                    else:
                        html = driver.page_source
                        break

                except Exception as e: # 네트워크 지연에 따른 예외 발생 시 대기 후 재실행
                    print(e)
                    time.sleep(3)
                    driver.find_element_by_css_selector('#review > li.list-group-item.btn-more > a').click()
                    time.sleep(1.5)

                    dates = driver.find_element_by_xpath(
                        "// *[ @ id = 'review'] / li[" + str(idx) + "] / div[1] / span[2]").text
                    if "전" in dates or "어제" in dates or int(dates.split("년")[0]) > 2019:
                        pass
                    elif int(dates.split("년")[0]) == 2019 and int(dates.split(" ")[1][:-1]) >= 3:
                        pass
                    else:
                        html = driver.page_source
                        break
                j += 1
                idx += 10

            # 전체 리뷰 확보 후 html 파싱    
            soup = BeautifulSoup(html, 'html.parser')
            
            # 리뷰만 가져오기 
            reviews = soup.find_all(class_='list-group-item star-point ng-scope')
            
            # 데이터 삽입 전 전처리
            for review in reviews:
                # 날짜 계산
                if "전" in review.find(class_="review-time ng-binding").get_text() or (
                        "어제" in review.find(class_="review-time ng-binding").get_text()):
                    year = 2021
                    month = datetime.date.today().month
                else:
                    year = int(review.find(class_="review-time ng-binding").get_text().split("년")[0])
                    month = int(review.find(class_="review-time ng-binding").get_text().split(" ")[1][0])

                # 리뷰     
                comment = review.find('p', attrs={'ng-show': 'review.comment'})
                comment = comment.get_text().replace("\n", " ").replace("  ", " ")
                
                # 주문 메뉴 
                menu = review.find('div', attrs={'class': "order-items default ng-binding"}).get_text()
                
                # 전체 평점 
                overall = review.find_all(class_="full ng-scope")
                
                # 세부 평점 
                points = review.find_all(class_="points ng-binding")
                try:
                    taste = int(points[0].get_text()) # 맛평점 
                except:
                    taste = ""
                try:
                    quantity = int(points[1].get_text()) # 양평점 
                except:
                    quantity = ""
                try:
                    delivery = int(points[2].get_text()) # 배달 평점
                except:
                    delivery = ""

                # 하나의 리뷰 데이터 구성     
                row = pd.DataFrame(
                    [(address.split(" ")[0], address.split(" ")[1], food_category, store_name, year, month,
                      len(overall), taste, quantity, delivery, comment, menu)],
                    columns=['시', '구', '업종명', '가게명', '년', '월', '전체평점', '맛 평점', '양 평점', '배달 평점', '리뷰 내용', '주문 내역'])
                
                # 최종 데이터셋에 추가
                df = df.append(row)

            driver.close()

    # 지나치게 많은 리뷰와 네트워크 지연 등의 문제로 동적 페이지 크롤링의 오류 다수 증가, 종료된 위치와 중간 결과를 저장함으로써 대처        
    except Exception as e:
        name = location.split(" ")[1]
        df_name = "{}_{}".format(name, food_category)
        df.to_csv("./{}_비정상종료.csv".format(df_name), encoding='utf-8-sig')
        print("종료위치 {} - {} - {}".format(food_category, urls.index(url), idx))
        print(e)


    df_name = "{}_{}".format(name, food_category)
    df.to_csv("./{}.csv".format(df_name), encoding='utf-8-sig')

### 수집을 위한 main 메소드
지역과 카테고리를 지정하며, 소요시간을 측정한다.

In [7]:
def main():
    categories = ["1인분주문","프랜차이즈", "치킨", "피자양식", "중국집", "한식", "일식돈까스", "족발보쌈", "야식", "분식", "카페디저트"]
    final_start = datetime.datetime.now()

    for category in categories:
        start = datetime.datetime.now()
        yogiyo_crawling('서울특별시 강남구 삼성동 16-1 강남구청', category)
        end = datetime.datetime.now()
        t = end - start
        hours, remainder = divmod(t.seconds, 3600)
        print("{} : {}시간 {}분".format(category, hours, remainder))

    final_end = datetime.datetime.now()

    t = final_end - final_start
    hours, remainder = divmod(t.seconds, 3600)
    print(final_start)
    print(final_end)
    print("전체소요시간 : {}시간 {}분".format(hours, remainder))

In [ ]:
main()

# 3. 요기요 가게정보 크롤링

각 리뷰별 데이터 뿐만 아니라, 추가 분석을 위한 각 가게별 평점, 메뉴 등 전반적인 가게 정보를 수집한다. 해당 메소드에서 수집하는 데이터는 다음과 같다.

- 가게정보
    - 가게명
    - 총 평점
    - 리뷰 개수
    - 사장님 리뷰 개수 
    - 최소주문금액
    - 배달시간
    - 배달료
    - 맛 평점
    - 양 평점
    - 배달 평점
    - 영업시간
    - 위치
    - 세스코 유무
    
    
- 메뉴정보
    - 가게 : {메뉴명 : 가격} 형태로 가게의 메뉴 정보를 json 형식으로 저장한다.


### 라이브러리 import

In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from tqdm import tqdm_notebook
import pandas as pd
import time
import datetime
import json

### 가게 정보 크롤링

데이터 수집은 전체 가게 리스트에서 수집이 한번에 가능한 정보와 가게별로 페이지에 접속하여 수집한 정보들로 구분된다. 이에 스크롤 조작을 통해 전체 가게 리스트를 수집하고, 기존에 수집한 URL 기반으로 가게별 페이지에 접근하여 이외의 정보를 수집한다.

In [9]:
def yogiyo_crawling_store_info(address, food_category):

    list_url = "https://www.yogiyo.co.kr/mobile/#/"

    # 주소 기준으로 초기화
    #driver = webdriver.Chrome('chromedriver')
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(4)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(address)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(1)

    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(1)

    # 초기 데이터 프레임 생성
    df = pd.DataFrame(columns=['가게명', '총평점', '리뷰개수', '사장님리뷰개수', '최소주문금액', '배달시간','배달료', 
                               '맛 평점', '양 평점', '배달 평점', '영업시간', '위치', '세스코유무'])

    # 해당 카테고리에 접속하여 마지막까지 스크롤
    driver.get(list_url + food_category)
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:

        scroll_down = 0
        while scroll_down < 10:
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            scroll_down += 1

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("끝")
            break

        last_height = new_height
    
    # 모든 가게의 정보를 담고 있는 html 파싱 
    main_html = driver.page_source
    soup = BeautifulSoup(main_html, 'html.parser')
    
    #가게 이름 리스트
    store_name = soup.find_all(attrs={'class':'restaurant-name ng-binding'})
    store_name = list(map(lambda x: x.text, store_name))    
    
    #총평점 리스트
    star_review = soup.find_all(attrs={'ng-show':'restaurant.review_avg > 0'})
    star_review = list(map(lambda x: float(x.text[-3:]), star_review))  
    
    #리뷰개수 리스트 
    review_count = soup.find_all(attrs={'ng-show':'restaurant.review_count > 0'})
    review_count = list(map(lambda x: int(x.text[30:-23]), review_count))  
    
    #사장님 댓글 갯수 리스트
    owner_count = soup.find_all(attrs={'ng-show':'restaurant.owner_reply_count > 0'})
    owner_count = list(map(lambda x: int(x.text[33:-23]),owner_count))  

    #최소 주문 금액 리스트
    min_price = soup.find_all(attrs={'class':'min-price ng-binding'})
    min_price = list(map(lambda x: int(x.text[:-7].replace(',','')), min_price))  

    #배달 시간 리스트
    delivery_time = soup.find_all(attrs={'ng-show':'restaurant.estimated_delivery_time'})
    delivery_time = list(map(lambda x: x.text[25:-23], delivery_time))  
    
    df['가게명'] = store_name
    df['총평점'] = star_review
    df['리뷰개수'] = review_count
    df['사장님리뷰개수'] = owner_count
    df['최소주문금액'] = min_price
    df['배달시간'] = delivery_time
    
    ### 가게정보 파싱 끝 ###

    ### 메뉴 및 세부 가게정보 가게별로 파싱###
    df.set_index('가게명', inplace = True)

    dic3 = {} 
    df2 = pd.read_csv("./"+address.split(" ")[1]+"url/"+address.split(" ")[1]+"_"+food_category+"_세부업종.csv")
    urls = df2['url'].tolist()
    
    # 가게별로 순회
    for i in range(len(urls)):
        try:
            driver.get(urls[i])
                      
            # 메뉴 크롤링
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            name = soup.find('span', attrs={'class' : "restaurant-name ng-binding"}).text
            sub_lists = soup.find_all('div', attrs={'class':'panel panel-default ng-scope', 'ng-repeat':"category in restaurant.menu"})
            sub_lists = sub_lists[1:]

            dic2 = {}
            for l in sub_lists:
                menu_category = l.find('span', attrs={'ng-class':'get_menu_class(category.slug)'}).text
                menus = l.find_all('td', attrs={'class' : 'menu-text'})
                for menu in menus:
                    menu_name = menu.find('div', attrs={'class' : 'menu-name ng-binding'}).text
                    menu_price = menu.find('span', attrs={'ng-bind' : 'item.price|krw'}).text
                    dic2[menu_name] = menu_price
            dic3[name] = dic2

            # 평점 크롤링
            review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            driver.execute_script("arguments[0].click();", review)
            time.sleep(1)
            taste_star = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[1]/div/ul/li[1]/span[2]/span[6]').text
            df.loc[name,'맛 평점'] = taste_star
            quantity_star = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[1]/div/ul/li[2]/span[2]/span[6]').text
            df.loc[name,'양 평점'] = quantity_star
            delivery_star = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[1]/div/ul/li[3]/span[2]/span[6]').text
            df.loc[name,'배달 평점'] = delivery_star

            # 가게 세부 정보 크롤링
            info = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]/a')
            driver.execute_script("arguments[0].click();", info)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            infos = soup.find_all(attrs={'class':'info-item'})
            for info in infos:
                if "업체정보" in info.text:
                    try:
                        delivery_fee = soup.find('span', attrs={'class': "list-group-item clearfix text-right ng-binding"}).text 
                    except:
                        delivery_fee = 0
                      
                    print(delivery_fee)
                    df.loc[name,'배달료'] = delivery_fee
                      
                    is_cesco = 0 if info.find('p', attrs={'ng-show':'restaurant.tags.length > 0 && restaurant.tags.indexOf("CESCO") >= 0',
                                                          'class':'ng-hide'}) is not None else 1
                    print(is_cesco)
                    df.loc[name,'세스코유무'] = is_cesco
                    opening_hour = info.find('p').find('span').text if "영업시간" in info.find('p').text else ""
                    print(opening_hour)
                    df.loc[name,'영업시간'] = opening_hour
                    location = info.find('p', attrs={'ng-show':"restaurant.address.length > 0"}).find('span').text
                    print(location)
                    df.loc[name,'위치'] = location
                    break

            time.sleep(3)

        except:
            continue # url은 보유하고있으나 이후 가맹 해지된 업장 예외처리
    
    # json, csv 저장
    json_file = './'+ address.split(" ")[1]+'_'+food_category+'_메뉴정보.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(dic3, f, ensure_ascii = False)
    
    df.to_csv('./'+ address.split(" ")[1]+'_'+food_category+'_가게정보.csv')

### 메소드 실행

In [ ]:
yogiyo_crawling_store_info('서울특별시 강남구 삼성동 16-1 강남구청', "족발보쌈")